In [1]:
import numpy as np
from scipy.linalg import expm

from functools import reduce

from qnl_projects.BHDecoder import qutrit_utils as qutils
from qnl_analysis import SimTools as ST

import matplotlib.pyplot as plt
%matplotlib inline

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


## Converting between a CSUM and the Diagonal Unitaries

There are two different diagonal unitaries which can be mapped to a CSUM with local unitaries.  Either we can:

* put $2\pi/3$ on the states $|11\rangle$ and $|22\rangle$, and $4\pi/3$ on the states $|12\rangle$ and $|21\rangle$, or
* put $4\pi/3$ on the states $|11\rangle$ and $|22\rangle$, and $2\pi/3$ on the states $|12\rangle$ and $|21\rangle$

In [11]:
desired_phases_1 = np.array([0,0,0,0,4,2,0,2,4])*np.pi/3
desired_phases_2 = np.array([0,0,0,0,2,4,0,4,2])*np.pi/3

U_ZZ_1 = np.diag(np.exp(1j*desired_phases_1))
U_ZZ_2 = np.diag(np.exp(1j*desired_phases_2))

In [12]:
H1 = np.kron(np.eye(3), qutils.qutrit_mapping['Hadamard'])
H1dag = np.kron(np.eye(3), qutils.qutrit_mapping['HadamardDag'])

H0 = np.kron(qutils.qutrit_mapping['Hadamard'], np.eye(3))
H0dag = np.kron(qutils.qutrit_mapping['HadamardDag'], np.eye(3))

$U_{ZZ}^1$ can be converted into a CSUM by sandwitching it with a Hadamard on the target qubit.  That is, the sequence ($H_1^\dagger$, $U_{ZZ}^1$, $H_1$) does a CSUM where Q0 is the control qubit and Q1 is the target qubit. 

In [19]:
np.round(np.abs(reduce(np.dot, [H1,U_ZZ_1, H1dag])))

array([[1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0.]])

Similarly, if we want to do a CSUM in the direction where Q0 is the target qubit and Q1 is the control qubit, then we should just apply the sequence ($H_0^\dagger$, $U_{ZZ}^1$, $H_0$)

In [20]:
np.round(np.abs(reduce(np.dot, [H0, U_ZZ_1, H0dag])))

array([[1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0.]])

We can also do a CMIN.  The sequence ($H_1$, $U_{ZZ}^1$, $H_1^\dagger$) does a CMIN where Q0 is the control qubit and Q1 is the target qubit. 

In [21]:
np.round(np.abs(reduce(np.dot, [H1dag,U_ZZ_1, H1])))

array([[1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0.]])

And similarly, to do a CMIN in the reverse direction, we just do ($H_0$, $U_{ZZ}^1$, $H_0^\dagger$)

In [23]:
np.round(np.abs(reduce(np.dot, [H0dag,U_ZZ_1, H0])))

array([[1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0.]])

We can also do similar things using the other diagonal unitary, which we called $U_{ZZ}^2$

In [24]:
np.round(np.abs(reduce(np.dot, [H1,U_ZZ_2, H1dag])))

array([[1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0.]])

In [25]:
np.round(np.abs(reduce(np.dot, [H0,U_ZZ_2, H0dag])))

array([[1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0.]])

In [26]:
np.round(np.abs(reduce(np.dot, [H1dag,U_ZZ_2, H1])))

array([[1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0.]])

In [27]:
np.round(np.abs(reduce(np.dot, [H0dag,U_ZZ_2, H0])))

array([[1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0.]])

### Defining alphas and the ZZ Hamiltonian

In [87]:
Q0Q1alphas = {'11': -0.27935 * 2*np.pi , 
              '12': 0.1599 * 2*np.pi,
              '21': -0.52793 * 2*np.pi,
              '22': -0.742967 * 2*np.pi}

Q6Q7_ZZdict = {'Q6GE_Q7inE': -0.2777, 'Q6GE_Q7inF': -0.6247, 'Q6EF_Q7inE': 0.5164, 'Q6EF_Q7inF': -0.114, 
        'Q7GE_Q6inE': -0.2743, 'Q7GE_Q6inF': 0.249, 'Q7EF_Q6inE': -0.3586, 'Q7EF_Q6inF': -0.9935} # measured values

Q6Q7alphas = qutils.get_ZZ_parameters(qutrit_0 = 6, qutrit_1 = 7, measured_ZZ_dictionary = Q6Q7_ZZdict)[0]
for key, val in Q6Q7alphas.items():
    Q6Q7alphas[key] = val*2*np.pi

def U_zz(evolution_time, alphas):
    H_zz = np.diag([0,0,0,0,alphas['11'], alphas['12'], 0, alphas['21'], alphas['22']])
    return expm(1j*H_zz*evolution_time)

## Version 0 - Only using EF pi pulses

In [83]:
Q0Q1_v0_times = {'T1': 0.61435267, 'T2': 0.10489224, 'T3': 0.61435267, 'T4': 0.10489224}
Q6Q7_v0_times = {'T1': 0.09199338, 'T2': 0.61632224, 'T3': 0.09199338, 'T4': 0.61632224}

In [84]:
# define the single-qutrit operations we need
Pi_EF = qutils.qutrit_mapping['EFX180']
Pi_EFm = np.conj(Pi_EF)

Pi_EF_0 = np.kron(Pi_EF, np.eye(3))
Pi_EFm_0 = np.kron(Pi_EFm, np.eye(3))

Pi_EF_1 = np.kron(np.eye(3), Pi_EF)
Pi_EFm_1 = np.kron(np.eye(3), Pi_EFm)

In [85]:
Uphi01_v0_4224= reduce(np.dot, [Pi_EF_1, U_zz(Q0Q1_v0_times['T4'], Q0Q1alphas), 
                       Pi_EF_0, U_zz(Q0Q1_v0_times['T3'], Q0Q1alphas), 
                       Pi_EFm_1, U_zz(Q0Q1_v0_times['T2'], Q0Q1alphas), 
                       Pi_EFm_0, U_zz(Q0Q1_v0_times['T1'], Q0Q1alphas)])


Uphi01_v0_2442 = reduce(np.dot, [Pi_EF_1, U_zz(Q0Q1_v0_times['T3'], Q0Q1alphas), 
                       Pi_EF_0, U_zz(Q0Q1_v0_times['T4'], Q0Q1alphas), 
                       Pi_EFm_1, U_zz(Q0Q1_v0_times['T1'], Q0Q1alphas), 
                       Pi_EFm_0, U_zz(Q0Q1_v0_times['T2'], Q0Q1alphas)])


In [89]:
Uphi67_v0_4224= reduce(np.dot, [Pi_EF_1, U_zz(Q6Q7_v0_times['T4'], Q6Q7alphas), 
                       Pi_EF_0, U_zz(Q6Q7_v0_times['T3'], Q6Q7alphas), 
                       Pi_EFm_1, U_zz(Q6Q7_v0_times['T2'], Q6Q7alphas), 
                       Pi_EFm_0, U_zz(Q6Q7_v0_times['T1'], Q6Q7alphas)])

Uphi67_v0_2442 = reduce(np.dot, [Pi_EF_1, U_zz(Q6Q7_v0_times['T3'], Q6Q7alphas), 
                       Pi_EF_0, U_zz(Q6Q7_v0_times['T4'], Q6Q7alphas), 
                       Pi_EFm_1, U_zz(Q6Q7_v0_times['T1'], Q6Q7alphas), 
                       Pi_EFm_0, U_zz(Q6Q7_v0_times['T2'], Q6Q7alphas)])

[ 0.          0.          0.          0.         -2.09439508  2.09439512
  0.          2.09439512 -2.09439508]
[ 0.          0.          0.          0.          2.09439512 -2.09439508
  0.         -2.09439508  2.09439512]


## Version 1 - Dynamically-decoupled from neighbors

In [59]:
Q0Q1_Delta_t = 0.59644 # us


#define the single-qutrit operations we need 
X = np.array([[0,1,0],
             [0,0,1],
             [1,0,0]])

Pi_GE = qutils.qutrit_mapping['X180']
EFZm90 = qutils.resulting_unitary([['EFZ-90']])

Pi_GE_nophi = np.dot(Pi_GE, EFZm90)*(1j)

XX = np.kron(X,X)

GEGE = np.kron(Pi_GE_nophi, Pi_GE_nophi)

#define the local Z gates which correct the phase on our state
Z1 = expm(-1j * 2*np.pi/3 * np.diag([0,1,1]))
Z2 = expm(-1j * 2*np.pi/3 * np.diag([0,1,1]))

total_Z = np.kron(Z1, Z2)

In [62]:
U_phi_dd = np.exp(1j*2.55411798)*reduce(np.dot, reversed([U_ZZ, XX, U_ZZ, XX, U_ZZ, GEGE, U_ZZ, XX, U_ZZ, XX, U_ZZ, GEGE, total_Z]))
U_CSUM_dd = reduce(np.dot, reversed([H1, U_phi_dd, H1dag]))